## 라마인데스로 RAG 구현하기

In [ ]:
import os
from datasets import load_dataset

# API KEY 설정
os.environ["OPENAI_API_KEY"] = "Your OpenAI API key"

# 데이터셋 다운로드
dataset = load_dataset('klue', 'mrc', split='train')
dataset[0]

/home/woosungkim/anaconda3/envs/deep_learning/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

In [ ]:
from llama_index.core import Document, VectorStoreIndex

text_list = dataset[:100]['context']
documents = [Document(text=text) for text in text_list]

# 인덱스 만들기
# OpenAI text-embedding-ada-002 모델 사용
# in-memory 방식의 벡터 데이터베이스 사용
index = VectorStoreIndex.from_documents(documents)

In [ ]:
print(dataset[10]['question']) # 현재 중국에서 신재생에너지 발전 용량이 차지하는 비율은?

retrieval_engine = index.as_retriever(similarity_top_k=5, verbose=True)
response = retrieval_engine.retrieve(
    dataset[10]['question']
)

print(len(response))
print(response[0].node.text)

현재 중국에서 신재생에너지 발전 용량이 차지하는 비율은?
4
“앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기후변화 관련 전문 투자 사모펀드인 ACP(Asia Climate Partners)의 아난드 프라카시 전무는 13일 ASK 2015 연사로 나와 “세계 1, 2위 인구 대국인 중국과 인도의 도시 이주자가 5억명에 달해 하수도나 전력, 공기 정화 등을 위한 관련 투자가 급증할 것”이라며 이같이 말했다.그는 지난 10년간 통신 관련 인프라 투자가 경제 성장을 이끈 것처럼 앞으로는 친환경과 신재생에너지 투자가 성장 흐름을 바꾸는 ‘게임체인저’가 될 것이라고 내다봤다.중국은 만성적인 스모그 등 환경오염 문제를 해결하기 위해 2020년까지 전체 전력 생산량의 15%를 신재생에너지로 충당할 계획이다. 지난해 중국에서 건설된 신규 발전소 가운데 신재생에너지 발전 용량이 차지하는 비중은 32%에 달했다.프라카시 전무는 “지난해 세계 신재생에너지 투자액은 전년보다 16% 증가한 3100억달러에 달했다”며 “2030년에는 전체 에너지 생산의 60%를 신재생에너지가 담당할 것”이라고 설명했다. 그는 태양열발전소와 풍력발전 분야에만 앞으로 5년 동안 2000억달러(약 220조원)의 투자가 이뤄질 것으로 전망했다. 이 중 “수익률이 높지 않은 유럽 시장보다는 아시아 시장을 겨냥해야 한다”는 게 그의 조언이다.환경오염이 심한 지역을 중심으로 세제 등 정부의 정책적 지원이 늘어나는 점도 주목했다. 그는 “기후변화와 관련한 투자는 각 국가 정책의 선택이 아닌 필수가 되고 있다”며 “발전 차액 지원 제도 등이 시행되면 수익률은 높아질 수밖에 없다”고 강조했다. 특히 폐기물 재활용이나 신선물류(콜드체인) 시스템 등은 발전 가능성이 크다고 예상했다. ACP는 아시아개발은행(ADB)과 투자사인 오릭스, 로베코가 아시아 전반의 기후변화 관련 투자를 위해 4억달러를 모아 설립한 회사다.


In [ ]:
# gpt-3.5-turbo를 기본 언어 모델로
query_engine = index.as_query_engine(similarity_top_k=1)
response = query_engine.query(
    dataset[10]['question']
)
print(response)

32%


In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# 검색을 위한 Retriver 생성
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1
)

# 검색 결과를 질문과 결합하는 synthesizer 
response_synthesizer = get_response_synthesizer()

# 위의 두 요소를 결합해 쿼리 엔진 생성
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

# RAG 수행
response = query_engine.query(dataset[10]['question']) # 현재 중국에서 신재생에너지 발전 용량이 차지하는 비율은?
print(response)
# 32%